In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import time

In [ ]:
# prompt: import drive module and create variable for drive path and also mount it

from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Get the path to your data directory
data_path = '/content/drive/My Drive/'


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_dataset(folder_path):
  images = []
  labels = []

  for label_folder in os.listdir(dataset_folder):
    if(label_folder == "Fire"):
      label = 1
    else:
      label = 0
    label_path = os.path.join(dataset_folder, label_folder)
    for image_file in os.listdir(label_path):
        try:
            image_path = os.path.join(label_path, image_file)
            img = cv2.imread(image_path)
            img = cv2.resize(img, (224, 224))
            img = img / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(label)
        except:
            print(f"Error loading image: {image_path}")
    # images = []
    # labels = []

    # for label in os.listdir(folder_path):
    #     print(label)
    #     label_path = os.path.join(folder_path, label)
    #     print(label_path)
    #     if os.path.isdir(label_path):
    #         for image_file in os.listdir(label_path):
    #             print(image_file)
    #             image_path = os.path.join(label_path, image_file)
    #             img = cv2.imread(image_path)
    #             img = cv2.resize(img, (224, 224))
    #             img = img / 255.0  # Normalize pixel values
    #             images.append(img)
    #             labels.append(label)  # Assuming labels are folder names (0 or 1)

  return np.array(images), np.array(labels)


In [ ]:
def detect_fire(image_path):
    # Load the trained model
    model = tf.keras.models.load_model(data_path+'model1/'+'fire_detection_model.h5')

    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalize pixel values
    img = np.reshape(img, (1, 224, 224, 3))  # Add batch dimension

    # Make prediction
    prediction = model.predict(img)

    # Display result
    if prediction[0][0] > 0.8:
        print("Fire detected!")
    else:
        print("No fire detected.")


In [ ]:
dataset_folder = data_path+'Fire Dataset/'

In [ ]:
images, labels = load_dataset(dataset_folder)
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


Error loading image: /content/drive/My Drive/Fire Dataset/Fire/F_921.jpg
Error loading image: /content/drive/My Drive/Fire Dataset/Fire/F_922.jpg
Error loading image: /content/drive/My Drive/Fire Dataset/Fire/F_923.jpg
Error loading image: /content/drive/My Drive/Fire Dataset/Fire/F_924.jpg
Error loading image: /content/drive/My Drive/Fire Dataset/Fire/F_94.jpg
Error loading image: /content/drive/My Drive/Fire Dataset/Fire/F_93.jpg


In [ ]:
print(len(train_images),len(test_images))
print(labels)

180 45
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [ ]:
print(train_labels.dtype)
print(test_labels.dtype)

int64
int64


In [ ]:
train_labels = tf.cast(train_labels, tf.float32)
test_labels = tf.cast(test_labels, tf.float32)

In [ ]:
print(train_labels.dtype)
print(test_labels.dtype)

<dtype: 'float32'>
<dtype: 'float32'>


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
6/6 [==============================] - 21s 3s/step - loss: 6.4838 - accuracy: 0.5611 - val_loss: 2.1667 - val_accuracy: 0.3556
Epoch 2/10
6/6 [==============================] - 18s 3s/step - loss: 0.5718 - accuracy: 0.7667 - val_loss: 0.7295 - val_accuracy: 0.8000
Epoch 3/10
6/6 [==============================] - 18s 3s/step - loss: 0.2283 - accuracy: 0.9278 - val_loss: 0.3089 - val_accuracy: 0.8889
Epoch 4/10
6/6 [==============================] - 20s 3s/step - loss: 0.1327 - accuracy: 0.9556 - val_loss: 0.5151 - val_accuracy: 0.7778
Epoch 5/10
6/6 [==============================] - 19s 3s/step - loss: 0.2385 - accuracy: 0.9278 - val_loss: 0.4069 - val_accuracy: 0.8889
Epoch 6/10
6/6 [==============================] - 18s 3s/step - loss: 0.1538 - accuracy: 0.9500 - val_loss: 0.4503 - val_accuracy: 0.8222
Epoch 7/10
6/6 [==============================] - 18s 3s/step - loss: 0.1318 - accuracy: 0.9556 - val_loss: 0.3698 - val_accuracy: 0.9111
Epoch 8/10
6/6 [==================

In [ ]:
model.save(data_path+'model1/'+'fire_detection_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
detect_fire("/content/Fire.jpg")

1/1 [==============================] - 0s 112ms/step
Fire detected!










